In [ ]:
from fastapi import FastAPI, Request, HTTPException
import hmac
import hashlib
import json
import os

app = FastAPI()

# Defina a chave HMAC compartilhada com a BMP
HMAC_SECRET_KEY = os.getenv("HMAC_SECRET_KEY", "sua_chave_hmac_aqui")

def verify_hmac_signature(body: dict, received_signature: str) -> bool:
    """
    Verifica a assinatura HMAC da requisição recebida.
    """
    body_str = json.dumps(body, separators=(',', ':'), ensure_ascii=False)
    calculated_signature = hmac.new(
        HMAC_SECRET_KEY.encode(), body_str.encode(), hashlib.sha256
    ).hexdigest()
    
    return hmac.compare_digest(calculated_signature, received_signature)

@app.post("/handshake")
async def handshake(request: Request):
    """
    Endpoint de Handshake para validar transações recebidas da BMP.
    """
    try:
        headers = request.headers
        body = await request.json()

        # Obtém a assinatura HMAC enviada pela BMP
        received_signature = headers.get("X-HMAC-Signature")
        if not received_signature:
            raise HTTPException(status_code=400, detail="HMAC signature missing")

        # Verifica a assinatura HMAC
        if not verify_hmac_signature(body, received_signature):
            raise HTTPException(status_code=403, detail="Invalid HMAC signature")

        # Se passou na verificação, retorna sucesso
        return {"status": "200 OK", "message": "Handshake validado"}

    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

